# Lab 5 - Localization and Mapping
## Introduction
Determining a robot’s orientation and position in a known environment, also known as localization, is a critical problem in the field of robotics. As is common in robotics, this seemingly simple problem is surprisingly difficult, and remains an active research area. In this lab, you will solve robotic localization by implementing Monte Carlo Localization (aka MCL, particle filter, or recursive bayes filter).

This lab consists of three parts, the first two of which must be submitted individually:

-   **Part A** - (Writing Assignment) Understand the motion and sensor model.
-   **Part B** - (Programming Assignment) Develop and test the particle filter algorithm in simulation.  
-   **Part C** - Demonstrate the functionality of your algorithm on the real RACECAR;   
&emsp;&emsp;&emsp;&emsp;- Configure and run **Google Cartographer** on the real RACECAR.

<img src="figures/pf.png" width="600">

## Submission 
<a id='submission'></a>
**Part A** - Submit your numeric results to writing assignment on **gradescope**. Include all derivations and formulas that you use to receive full credit.

**Part B** - Submit your solutions to programming assignment on **gradescope**. After running the automated tests described at the  end of this 			lab, you will generate a file called  `log.npz`.  **You must upload this to gradescope to receive credit**.

Parts A & B are due on **Wednesday, March 20, 2019**, which is 1 week after the release date of this lab.

**Part C** - Run your implementation of MCL from part B on the real RACECAR and present your results to teaching staff. The things we will look into:
   -   Numerical evidence that your algorithm is working in the form of charts/data
		-   Convergence rates, cross track error, etc
		-   The simulator odometry is perfectly accurate, so that can be your ground truth!
   -   An [illustrative video](https://www.youtube.com/watch?v=-c_0hSjgLYw&t=6s) of your particle filter working in a simulation environment, overlaying:
		-  Visualization of inferred position
		-   Visualization of the particle distribution
		-   The known map
		-   Laser scan data in the coordinate frame of your inferred position (it should align fairly well with the walls in the known map)
		-   Any other byproducts of your algorithm which you find worthy of visualization
   -   A [similarly illustrative video of your car localizing](https://www.youtube.com/watch?v=-c_0hSjgLYw) in the Stata basement environment

This lab will **be presented alongside lab 6** which is due on **April 3, 2019**.



## Part A - Writing Assignment
Work on the following questions and submit your answers to autograder. These questions will help you understand the algorithm before diving into coding. 

<a id='question1'></a>
**Question 1**. (**Motion Model**) Consider a deterministic motion model based on odometry information. The motion model takes as arguments the old particle pose, as well as the last odometry data, and return a new pose with the odometry “applied” to the old poses. 
$$
\Delta \mathbf{x}:=[\Delta x, \Delta y, \Delta\theta]^T=f(u_t)
$$
$$
\mathbf{x}_t = g(\mathbf{x}_{t-1}, \Delta x)
$$
where $f(\cdot)$  takes the control input $u_t$ and returns the changes in pose, $g(\mathbf{x}_{t-1}, \Delta x)$ is the motion model applying the observed change $\Delta \mathbf{x}:=[\Delta x, \Delta y, \Delta \theta]^T$ in pose to the previous robot's pose $\mathbf{x}_{t-1}$ to get predictions of the next pose $\mathbf{x}_t$. 

  **i**. Let the poses estimated by odometry are $\mathbf{\hat{x}}_{t-1} = [0, 0, \pi/6]^T$ and $\mathbf{\hat{x}}_{t} = [0.2, 0.1, 11\pi/6]^T$ at time $t-1$ and $t$, respectively. Both are expressed in the world coordinate system. Find the estimated change of pose $\Delta \mathbf{x}$ between time $t-1$ and $t$ expressed in the body frame.

  **ii**. Odometry data is accumulated via dead reckoning, therefore it is very inaccurate. Now assume the previous pose estimated by the particle filter is $\mathbf{x}_{t-1} = [3,4,\pi/3]^T$ (in world frame). Find current pose $\mathbf{x}_t$ (in world frame) estimated based on the measurement from odometry in part (**i**).

<a id='question2'></a>
**Question 2**. (**Sensor Model**) The sensor model $p(z_{t}^{k}| x_{t}, m)$ defines how likely it is to record a given sensor reading $z_{t}^{k}$ from a hypothesis position $x_{t}$ in the map $m$ at time $t$. The definition of this likelihood is strongly dependent on the type of sensor used - a laser scanner in our case.

   Typically, there are a few cases to be modeled in determining $p(z_{t}^{k}| x_{t}, m)$:

	a.  Probability of detecting a known obstacle in the map
    b.  Probability of a short measurement, maybe due to unknown obstacles (cats, people, etc)
    c.  Probability of a missed measurement, usually due to a reflected LiDAR beam
    d.  Probability of a random measurement, maybe due to unexpected asteroid collisions
    
We typically represent (a) as a gaussian distribution around the ground truth distance between the hypothesis pose and the known map obstacle. Thus, if the measured range exactly matches the expected range, the probability is large. If the measured range is $z_{t}^{k}$ and the ground truth range is determined (via ray casting) to be $z_{t}^{k^{\ast}}$:

$$
	p_{hit}(z_{t}^{k}| x_{t}, m)  = \begin{cases}
	\frac{1}{\sigma\sqrt{2\pi}} e^{-\frac{(z_{t}^{k}-z_{t}^{k^{\ast}})^2}{2\sigma^2}}, & \text{if}~ 0 \leq z_{t}^{k} \leq z_{t}^{k^{\ast}}\\
	0, & \text{otherwise} 
	\end{cases}
$$
	
Case (b) is represented as a downward sloping line as the ray gets further from the robot. Intuitively, the likelihood of sensing unexpected objects decreases with range because we are more likely to detect obstacles nearby. This likelihood can be modeled as:
$$
	p_{short}(z_{t}^{k}| x_{t}, m) =  \begin{cases}
         \lambda_{short} (1 - \frac{z_{t}^{k}}{z_{t}^{k^{\ast}}}),   &   \text{if}~ 0 \leq z_{t}^{k} \leq z_{t}^{k^{\ast}}\\
         0,   &   \text{otherwise}
\end{cases}
$$

Case (c\) is represented by a large spike in probability at the maximal range value, so that reflected measurements do not significantly discount particle weights.
$$
	p_{max}(z_{t}^{k}| x_{t}, m)  =\begin{cases}
	1,  &  \text{if}~ z = z_{max}\\
	0,  &  \text{otherwise} 
	\end{cases}
	$$

Case (d) is represented by a small uniform value, to account for unforeseen effects.
	$$
	p_{rand}(z_{t}^{k}| x_{t}, m)  = \begin{cases}
	\frac{1}{z_{max}},  &  \text{if}~ 0\leq z_{t}^{k} < z_{max}\\
	0,                            & \text{otherwise} 
	\end{cases}
$$
	
These four different distributions are now mixed by a weighted average, defined by the parameters $\alpha_{hit}, \alpha_{short},\alpha_{max},\alpha_{rand}$:
$$
	 p(z_{t}^{k}| x_{t}, m)  = \alpha_{hit} \cdot p_{hit}(z_{t}^{k}| x_{t}, m)  + \alpha_{short} \cdot p_{short}(z_{t}^{k}| x_{t}, m)  + \alpha_{max} \cdot p_{max}(z_{t}^{k}| x_{t}, m)  + \alpha_{rand} \cdot p_{rand}(z_{t}^{k}| x_{t}, m) 
$$
	 
Given $\alpha_{hit} = 0.75$, $\alpha_{short}=0.01$,$\alpha_{max}=0.07$,$\alpha_{rand}=0.12$, $z_{max}=200$ px, $\sigma=8.0, \lambda_{short}=2.0$ and $z_{t}^{k^{\ast}} = 140$ px, find the probability  $p(z_{t}^{k}| x_{t}, m)$  when  
**i**.    $z_{t}^{k} = 0$

**ii**.   $z_{t}^{k} = 50$

**iii**.  $z_{t}^{k} = 100$

**iv**.   $z_{t}^{k} = 150$

**v**.    $z_{t}^{k} = 200$

## Part B - Programming Assignment
### 1. Getting Started
Grab (fork) the skeleton code from here: [https://github.mit.edu/2019-RSS/lab5_localization](https://github.mit.edu/2019-RSS/lab5_localization). In this lab, you will use RangeLibc for super fast ray casting. Check out [the repo](https://github.com/kctess5/range_libc) if for some reason you don’t already have it installed. An algorithm overview and usage instructions can be found here: [RangeLibc documentation](https://docs.google.com/document/d/1NSRAAZhNbmrsyQynYBgQuT2EUYupjg3Qafu1OtkTYtc/export?format=pdf).

### 2. Implementation


#### 2.1 Motion Model
While you could use the IMU in the motion model, we recommend using the wheel odometry coming from dead-reckoning integration of motor and steering commands. It will probably be less noisy than the IMU except in extreme operating conditions. You may assume that the motor commands relate to the movement of your car according to the [geometric Ackermann model](http://www.ri.cmu.edu/pub_files/2009/2/Automatic_Steering_Methods_for_Autonomous_Automobile_Path_Tracking.pdf), with some noise due to tire slip and other imperfections. The `/vesc/odom` topic does this for you already on the car!

The precise definition of your motion model is up to you. You might empirically determine your noise coefficients based on what works, or could try to gather data from the car which allows you to directly determine your measurement uncertainty. You will also have to model how your available odometry data (i.e. steering commands speed and steering) correspond to changes in states $(x,y,\theta)$. 

You can start with implementing your algorithm in [Question 1 of part A](#question1).


#### 2.2 Sensor Model
Once you have updated particle positions via your motion model, you use the sensor model to assign likelihood weights to each particle. This allows good hypotheses to have better likelihood of being sampled on the next iteration, and visa versa for bad hypotheses. 

You might want to use equations listed in [Question 2 of part A](#question2) to get the likelihood weights of each particle. While it is not strictly necessary for your particle filter to function (since the normalization step reweights particles), ideally $p(z_{t}^{k}|x_t, m)$ integrates to 1, given that it is a probability. It can be tricky to define the sum of four terms such that it always adds to 1, so an easy way to handle this is to simply normalize the discretized model after precomputation (see below).
<img src="figures/sensor_model_slice2.png" width="600">
#### Precomputing the model
In section 2.2, you basically formulate a function to compute $p(z_{t}^{k}|x_t, m)$ which more or less encodes the likelihood of recording range measurement $z_{t}^{k}$, given the ground truth range $z_{t}^{k^{\ast}}$ (found with ray casting). This function could be used (along with ray casting) to directly evaluate the likelihood of recording your actual laser scan at each hypothesis pose. However, there’s a better way! 
Instead of evaluating the function millions of times at runtime, you can evaluate it over a discretized grid of its inputs ($z_{t}^{k}$ and $z_{t}^{k^{\ast}}$) at initialization time, and store a lookup table. This strategy is described in section 3.4.1 of [2]. The motivation is twofold:
1. Reading off a probability value from a table based on your actual sensor measurement and raycast ‘ground truth’ values is MUCH faster than doing all the math every time;
2. This provides a good opportunity to numerically normalize the $p(z_{t}^{k}|x_t, m)$ function.

Here’s some pseudocode for how this might look in your implementation:  

```python
def precompute_sensor_model()
    # Experimentally determine some constants in the sensor model
    # create a lookup table
    return sensor_model_table

sensor_model_table = precompute_sensor_model()  
def sensor_model_table_lookup(r, d):  
    '''
    Get the probability of measured distance r given groundtruth distance d from precomuted table
    ''' 
    return sensor_model_table[discretize(r), discretize(d)] 
```

And your sensor model might look something like:

```python
range_method = range_lib."SomeRangeMethod"(map_msg, max_range_in_pixels)

def sensor_model(px,py,theta,ranges):
    '''
    Find the probablistic weight for each particle in the proposal distribution
    
    This sensor model is defined for a single particle (px,py,theta)  
    however, in practice, you will find it more efficient to define your  
    sensor model for the entire array of particles instead 
    
    Args:
        px, py, theta: pose of the particle
        ranges: measured ranges from LiDAR
    '''
    # queries is the set of (x,y,theta) poses to ray cast from 
    queries = make_query_states(px, py, theta)  
    groundtruth_ranges = np.zeros(queries.shape[0])  
    range_method.calc_range_many(queries,groundtruth_ranges)  
    # compute weight
    partial_weights = map(lambda r,d: sensor_model_table_lookup(r,d), ranges, groundtruth_ranges)  
    weight = product(partial_weights)  
    # do other things to the weight...
    return weight

```

The above pseudocode for a sensor model is unnecessarily slow - think about how it could be improved. See below, and the documentation for RangeLibc for further discussion.
<img src="figures/sensor_model2.png" width="600">

#### 2.3 Particle Resampling
In the resampling step of your particle filter, you should randomly draw from the set of particles according to the probability weights computed in the previous iteration:
$$
    \mathbf{x}_{t-1}^{k} \sim \mathbf{X}_{t-1}
$$
This set is called the proposal distribution. Notice: you will very likely (à la [birthday paradox](https://en.wikipedia.org/wiki/Birthday_problem)) draw the same particle more than once, but this is ok because **your motion model incorporates randomness** that will ensure we don’t evaluate the observation model with several copies of the same particle.

You may find the [numpy.random.choice](https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.choice.html) function useful!

#### 2.4 Particle Filter
Once you have defined your motion and sensor models, you should implement the recursive algorithm something like: 

```python
def MCL(X_prev, u_prev, z_cur):  
    X_cur = {}  
    for i in range(m):  
        # sample a pose from the old particles according to old weights.  
        # Samples implicitly represent the prior prob. dist. Bel(x_prev)  
        x_prev = resample(X_prev)   
        # update the sampled pose according to the motion model 
        x_cur = motion_model(x_prev, u_prev) 
        # weight the updated pose according to the sensor model   
        w_cur = sensor_model(x_cur, z_cur) 
        # add the new pose and weight to the new distribution 
        X_cur = update(x_cur, w_cur)  
    # normalize weights, should sum to 1 
    X_cur = normalize_importance_weight(X_cur)  
    return X_cur

```

```python
def particle_filter():  
    X = Bel(x0) ← initial particles   
    while True:  
        u = get_last_odometry()    
        z = get_last_sensor_readings()   
        X = MCL(X, u, z)  
        # inferred pose ← expected value over particle distribution 
        pose = expectation(X)
```

### 3. **Test your implementation in simulation**.
 Similar to previous labs, we have provided an autograder to test your particle filter. The autograder will launch the following nodes:
-       `racecar/launch/teleop.launch` (temporarily at startup)
-   	`lab5_localization/launch/map_server.launch`
-   	`lab5_localization/launch/localize.launch`
    

Once the autograder detects your particle filter is running (as soon as it receives a message from `/pf/pose/odom`) it will launch a rosbag collected on a Hokuyo flavored RACECAR while driving around the basement. Then, the autograder receives messages published on `/pf/pose/odom` and extracts (time, x, y, theta) information from each message via `Odometry.pose.pose.position`, `quaternion_to_angle` (`Odometry.pose.pose.orientation`) and `Odometry.header.stamp`. When the test completes (or is terminated early) the autograder will output a `log.npz` file that contains this trajectory data, which you need to upload to Gradescope. You should all submit the log.npz file, but you can share that file among your own team.

Periodically throughout testing, we provide initialization via the `/initialpose `topic, so it’s extremely important that you implement this functionality in your solution. You are not allowed to manually provide your own reinitialization messages during testing.

You may run the test at less than real time via the `--rate` argument (e.g. `./run_tests --rate 0.5`). However, the grade you receive will also be scaled by this quantity according to the below chart, so it is in your interest to run at real time! For fairness, we require that all teams run the test scripts on their car. This will avoid potentially penalizing people with slower laptops.

FYI, TA's solution generally gets $>0.9$ points on Gradescope. Note: the bit where the car is by the bikes is tricky! The map isn’t perfect, and therefore you will not achieve perfection there. Don’t worry about it, everyone will have the same problem.

Note: you will have to install the map server node on the RACECAR.

	sudo apt-get install ros-kinetic-map-server
    
<img src="figures/scaling.png" width="600">

### 4. Notes and Tips
If you implement the particle filter exactly as described above, you will likely discover that it has very poor performance. Largely, this poor performance is due to assumptions made during the derivation of the particle filter which are not strictly true. 

For example, the particle distribution approximates the true distribution in the limit - with an infinite number of particles. Clearly this is problematic since we maintain at max a few thousand particles. You should take a look at [4,5] for some useful tips for improving real world problems. 

You will want to downsample the number of range measurements from the particle filter from >1000 to less than 100. This will make the probability distribution over your state space less “peaked” and increase the number of particles you can maintain in real time (less ray casting). Additionally, you will probably want to “squash” your sensor model output probability by raising it to a power of less than one (1/3 for example) to make your distribution even less peaked. If your are confused by this paragraph, look at [4,5]. 
Similarly, you may notice that the TA sensor model table shown above is not as peaked as you might expect given the LiDAR accuracy (namely the “hit” term in [question 2 of part A](#question2)) . This is largely to smooth the probability distribution over the particle state space so that the convergence basin of the algorithm is larger.

Here are more tips on how to improve your code:

#### 4.1 Writing efficient Python
Since the algorithm must run at >20Hz with a large number of particles, an efficient implementation is a requirement for success. There are a few tricks you can use, primarily:
-   Use numpy arrays for absolutely everything - python lists → slow

-   Use numpy functions on numpy arrays to do any computations, 
    - avoid Python for loops like the plague
    -   [Slice indexing is your (best) friend.](https://docs.scipy.org/doc/numpy/reference/arrays.indexing.html)
    
-   Use the smallest number of operations required to perform your arithmetic

-   Avoid unnecessary memory allocations
    -   Cache and reuse important numpy arrays by setting them to the right size during initialization of your particle filter as “self” variables
    -   Be careful! Avoid bugs due to stale state in reused variables!
    
-   Identify your critical code paths, and keep them clean
    - Conversely, don’t worry too much about code that is called infrequently
    
-   Push code to Cython/C++ if necessary
    -   You probably won’t need to do this much since RangeLibc already does this
    
-   Avoid excessive function calls - function call overhead in Python → slow

-   Use RangeLibc
    -   saveTrace with Bresenham’s Line can be useful for debugging (see docs)

-   Don’t publish visualization messages unless someone is subscribed to those topics
```
    if  self.[some_pub].get_num_connections() >  0 …
```
-   USE A PROFILER to identify good candidates for optimization
    -   [http://projects.csail.mit.edu/pr2/wiki/index.php?title=Profiling_Code_in_ROS](http://projects.csail.mit.edu/pr2/wiki/index.php?title=Profiling_Code_in_ROS)
-   Use rostopic hz to determine how fast your nodes are publishing.
    -   If your particle filter is publishing the pose significantly slower than the laser scan than there is a lot of lag.

#### 4.2 RangeLibc

Assuming a reasonably efficient architecture, you will find that the bottleneck computation in running your particle filter is the determination of ground truth ranges between hypothesis poses (particles) and the nearest obstacles in the map - ray casting. Rather than having you implement the “calc_range” function yourself in Python, we provide the RangeLibc library to enable real time particle filter operation. See [RangeLibc document](https://docs.google.com/document/d/1NSRAAZhNbmrsyQynYBgQuT2EUYupjg3Qafu1OtkTYtc/export?format=pdf) for an algorithm overview and usage instructions. We highly recommend you read through it since it includes tips to make your sensor model implementations significantly simpler and faster.

#### 4.3 RViz is your friend

-   Visual inspection is the best debugging tool
    
-   Significantly better than looking at a stream of numbers in a terminal
    
-   Add visualization early and often
    
-   Subscribe to the /initialpose or /clicked_point topics and use the associated tools in RViz (“2D Pose Estimate”/”Clicked Point”) to initialize or reinitialize your particle filter [(see video example of this)](https://www.youtube.com/watch?v=IQ1GtNJvBjg)

#### 4.4 Debugging tips

#### Motion model

1.  Disable your sensor model update
    
2.  Disable your motion model noise
    
3.  Ensure that when you move the car, the particles move in the same manner.
    
4.  Rotate your car but do not reinitalize your particles - once again drive around and ensure your particles do something reasonable
    
5.  Once you’re happy with the results from the above tests, enable your motion model noise. Now, you should see the particles spreading out as you drive around, but they should all move in roughly the right direction without excessive spread.
    
6.  Enable your sensor model now and see that the particle distribution ‘collapses’ around the correct pose after each sensor measurement is considered
    

#### Sensor model

1. Print your probabilities and look at them. The sensor model is prone to some numerical issues because multiplying a bunch of very small numbers leads to extremely small values which can exceed float precision. If this happens, you might be able to think of ways to rearrange your order of operations to avoid these problems.
    
2. Evaluate your model with custom arguments
    
  - Providing the same ‘observation’ and ‘ground truth’ scans should result in reasonably large probabilities
    
  - Providing two extremely different scans should result in low probabilities
    
  - Etc - make up some test cases and ensure your expectations match reality
    

#### Ray casting with RangeLibc

Try doing some simple cases of ray casting and make sure you get basically what you expect. You can (for example) dump the distance values you get from simulating a laser scan at a known pose, and plot them somehow. What you see in the plot should match your expectations, if not, maybe you have your x/y coordinates reversed, or some other problem like that.

## Part C - Run on Real RACECAR
1. Demonstrate the functionality of your algorithm on the real RACECAR; 
2. Configure and run **Google Cartographer** on the real RACECAR. [Cartographer](https://github.com/googlecartographer/cartographer) is a system that provides real-time simultaneous localization and mapping ([SLAM](https://en.wikipedia.org/wiki/Simultaneous_localization_and_mapping)) in 2D and 3D across multiple platforms and sensor configurations.  Please go to this repository, follow the instruction to configure and run cartographer on the real RACECAR.

	    https://github.mit.edu/2018-RSS/cartographer_config
3. Check [Submission](#submission) for further instruction.

We will announce the schedule for  the presentation in next lab.

## References

1. <a name="ThrunRobust"></a>[S. Thrun, D. Fox, W. Burgard and F. Dellaert. “Robust Monte Carlo Localization for Mobile Robots.” Artificial Intelligence Journal. 2001](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.71.6016&rep=rep1&type=pdf).
2. <a name="FoxMarkov"></a>[D. Fox, W. Burgard, and S. Thrun. “Markov localization for mobile robots in dynamic environments,” Journal of Artificial Intelligence Research, vol. 11, pp. 391427, 1999](http://www.jair.org/media/616/live-616-1819-jair.pdf).
3. <a name="FoxAdvances"></a>[D. Fox. “KLD-sampling: Adaptive particle filters,” Advances in Neural Information Processing Systems 14 (NIPS), Cambridge, MA, 2002. MIT Press](https://papers.nips.cc/paper/1998-kld-sampling-adaptive-particle-filters.pdf).
4. <a name="BagnellPracticle"></a> [D. Bagnell “Particle Filters: The Good, The Bad, The Ugly”
](http://www.cs.cmu.edu/~16831-f12/notes/F14/16831_lecture05_gseyfarth_zbatts.pdf)
5. <a name="BootsImportance"></a>[B. Boots “Importance Sampling, Particle Filters”](https://web.archive.org/web/20170209092754/http://www.cc.gatech.edu/~bboots3/STR-Spring2017/Lectures/Lecture4/Lecture4_notes.pdf)
6. <a name="WalshCDDT"></a>[C. Walsh and S. Karaman “CDDT: Fast Approximate 2D Ray Casting for Accelerated Localization”](https://arxiv.org/abs/1705.01167)